#  **Préparation des Données Météorologiques et des caractéristiques nécessaires**
---
**Dans ce notebook, nous avons effectué plusieurs étapes essentielles de prétraitement et d’enrichissement des données météorologiques, avec l’objectif d’améliorer leur qualité et d’extraire des insights utiles pour une analyse approfondie ou une modélisation prédictive.**



# 0️⃣ **Chargement & Compréhension des données**





In [ ]:
import pandas as pd
df = pd.read_csv("C:/Users/salma souissi/Downloads/data_all_cities_hourly.csv")

In [ ]:
#taille des données
print(df.shape)

(1042268, 14)


In [ ]:
print(df.head())

   temp  dwpt  rhum  prcp  snow   wdir  wspd  wpgt    pres  tsun  coco  \
0  16.3   2.3  39.0   NaN   NaN  260.0   7.6   NaN  1023.8   NaN   2.0   
1  15.0   1.1  39.0   NaN   NaN  270.0  13.0   NaN  1023.0   NaN   2.0   
2  15.0  -1.2  33.0   NaN   NaN  260.0  11.2   NaN  1023.0   NaN   2.0   
3  14.1  -1.1  35.0   NaN   NaN  150.0   1.8   NaN  1023.3   NaN   2.0   
4  13.0   1.0  44.0   NaN   NaN  180.0   9.4   NaN  1022.0   NaN   2.0   

         date  hour   city  
0  2020-02-24    18  Tunis  
1  2020-02-24    19  Tunis  
2  2020-02-24    20  Tunis  
3  2020-02-24    21  Tunis  
4  2020-02-24    22  Tunis  


In [ ]:
print(df.columns)

Index(['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres',
       'tsun', 'coco', 'date', 'hour', 'city'],
      dtype='object')


In [ ]:
print(df['city'].unique())

['Tunis' 'Ariana' 'Ben Arous' 'Manouba' 'Nabeul' 'Zaghouan' 'Bizerte'
 'Beja' 'Jendouba' 'Kef' 'Siliana' 'Kasserine' 'Sidi Bouzid' 'Kairouan'
 'Kébili' 'Tozeur' 'Gabès' 'Medenine' 'Tataouine' 'Mahdia' 'Monastir'
 'Sousse' 'Sfax' 'Gafsa']


# 1️⃣**Nettoyage et réorganisation des colonnes**

In [ ]:
# Renommer les colonnes
df.rename(columns={
    'temp': 'temperature',
    'prcp': 'precipitation',
    'wspd': 'wind_speed',
    'rhum':'humidity'
}, inplace=True)
#supprimer les colonnes non necessaires
df=df.drop(columns=['dwpt','snow','wdir','wpgt','pres','tsun','coco'])
# Nouveau ordre des colonnes
new_order = ['city', 'date', 'hour', 'temperature', 'precipitation', 'wind_speed','humidity']

# Réorganiser les colonnes
df = df[new_order]


print(df.head())

    city        date  hour  temperature  precipitation  wind_speed  humidity
0  Tunis  2020-02-24    18         16.3            NaN         7.6      39.0
1  Tunis  2020-02-24    19         15.0            NaN        13.0      39.0
2  Tunis  2020-02-24    20         15.0            NaN        11.2      33.0
3  Tunis  2020-02-24    21         14.1            NaN         1.8      35.0
4  Tunis  2020-02-24    22         13.0            NaN         9.4      44.0


In [ ]:
# Remplacer les valeurs NaN par 0
df.fillna(0, inplace=True)

 ## **Installation de la bibliothèque holidays**

---

🔹 **holidays est une bibliothèque Python qui permet de récupérer facilement les jours fériés d’un pays donné. Elle est particulièrement utile dans les analyses de séries temporelles, notamment pour identifier l’impact des jours fériés sur certaines variables (ex : ventes, fréquentation, météo, etc.).**



In [ ]:
pip install holidays

  Obtaining dependency information for holidays from https://files.pythonhosted.org/packages/f8/8f/9cff125e50b56e29e7e05776dc74e56fc70b79830f0b85e947e5be831e96/holidays-0.67-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/820.7 kB ? eta -:--:--
   - -------------------------------------- 30.7/820.7 kB 1.3 MB/s eta 0:00:01
   -- ------------------------------------ 61.4/820.7 kB 656.4 kB/s eta 0:00:02
   ---- --------------------------------- 102.4/820.7 kB 653.6 kB/s eta 0:00:02
   ----- -------------------------------- 122.9/820.7 kB 722.1 kB/s eta 0:00:01
   -------- ----------------------------- 184.3/820.7 kB 794.9 kB/s eta 0:00:01
   --------- ---------------------------- 204.8/820.7 kB 731.4 kB/s eta 0:00:01
   ---------- --------------------------- 235.5/820.7 kB 758.5 kB/s eta 0:00:01
   ------------ ------------------------- 266.2/820.7 kB 780.5 kB/s eta 0:00:01
   -------------- ----------------------- 307.2/820.7 kB 759.5 kB/s eta 0:00:01
   -------


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\salma souissi\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [ ]:
import holidays
jours_feries = holidays.country_holidays('TN') #'TN' pour la Tunisie)



In [ ]:
# Convertir la colonne Date en datetime
df["date"] = pd.to_datetime(df["date"])


# 2️⃣**Enrichissement du dataset avec de nouvelles caractéristiques**

---

Dans cette section, nous ajoutons plusieurs colonnes pour capturer des informations temporelles et contextuelles qui pourraient influencer les tendances météorologiques et les comportements des consommateurs.





1.    **Moyenne mobile sur 7 jours** : Cette colonne permet d'ajouter une vision lissée des températures sur une semaine, ce qui est utile pour capturer des tendances et éviter les variations brutales d'un jour à l'autre.
## Ajout d’informations temporelles
2.   **Le mois 📅** : Cela permettra d’identifier les variations saisonnières.
3.   **Le jour de la semaine** : Cette colonne est utile pour observer les tendances hebdomadaires.
4.   **Le week-end 🏖️** :  ajouter une colonne binaire (1 si c'est le week-end, 0 sinon), car les comportements peuvent être différents en fin de semaine.
## Identification des jours spéciaux
5.  **Jour férié**  : Cette colonne est utile pour analyser l'impact des jours fériés sur les ventes ou les habitudes météorologiques.
6.  **Saison météorologique 🌞❄️** : Cela permet de différencier les comportements selon la saison.
7.  **Vacances scolaires 🏫** : Les périodes de vacances peuvent influencer la consommation et la météo perçue.
8.  **Mois du Ramadan 🕌** : Le Ramadan modifie fortement les habitudes de consommation, il est donc important de l’intégrer.
9.  **Nouvel Année 🎉** : Les festivités du Nouvel An peuvent influencer les tendances de consommation et de météo.
## Catégorisation des régions 🗺️
définition d'une classification des régions tunisiennes en trois catégories

* Côtières (ex. : Sousse, Monastir, Mahdia...)

* Intérieures (ex. : Kairouan, Gafsa, Tataouine...)

* Hors côtières (ex. : Tunis, Ariana, Ben Arous...)

‖‖ Cela permet d’analyser les différences entre les types de zones. ‖‖
## Saison touristique 🏖️
Nous définissons trois niveaux de saisonnalité pour le tourisme :

* Haute saison (mai à août)

* Moyenne saison (avril, septembre à novembre)

* Basse saison (janvier à mars, décembre)

‖‖ Cela est crucial pour analyser les variations du climat et des habitudes des consommateurs. ‖‖

---

**Toutes ces transformations permettent d'enrichir notre dataset avec des informations précieuses, facilitant une analyse plus fine des tendances climatiques et comportementales.**






In [ ]:


#ajouter une colonne indiquant le mois :
df["mois"] = df["date"].dt.month
#____________________
jours_semaine = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi","Samedi", "Dimanche"]
#Générer les jours en respectant l'ordre
df["Jour"] = [jours_semaine[i % 7] for i in range(len(df))]
#Ajouter la colonne Weekend (1 si Samedi ou Dimanche, sinon 0)
df["Weekend"] = df["Jour"].isin(["Samedi", "Dimanche"]).astype(int)
# Ajouter une colonne indiquant si la date est un jour férié
df['jour_ferie'] = df['date'].apply(lambda x: x in jours_feries).astype(int)
# Ajouter une colonne "saison"
df["saison"] = df["date"].apply(lambda x: "été" if x.month in [6, 7, 8] else "hiver" if x.month in [12, 1, 2] else "printemps/automne")
#vacances scolaires :
vacances_scolaires = [('2020-12-15', '2021-01-01'), ('2021-03-14', '2021-03-27'),('2021-06-30','2021-09-14'),('2021-12-15', '2022-01-01'), ('2022-03-14', '2022-03-27'),('2022-06-30','2022-09-14'),('2022-11-15', '2023-01-1'), ('2023-03-14', '2023-03-27'),('2023-06-30','2023-09-14'),('2023-11-15', '2024-01-1'), ('2024-03-14', '2024-03-27'),('2024-06-30','2024-09-14')]
df['vacances_scolaires'] = df['date'].apply(
    lambda x: 1 if any(pd.to_datetime(start) <= x <= pd.to_datetime(end) for start, end in vacances_scolaires) else 0)
#ramadan :
ramadan_dates = {
    '2020': ('2020-04-23', '2020-05-23'),
    '2021': ('2021-04-13', '2021-05-12'),
    '2022': ('2022-04-02', '2022-05-01'),
    '2023': ('2023-03-23', '2023-04-22'),
    '2024': ('2024-03-11', '2024-04-09')
}
df['ramadan'] = df['date'].apply(
    lambda x: 1 if any(pd.to_datetime(start) <= x <= pd.to_datetime(end) for start, end in ramadan_dates.values()) else 0)
#new year:
new_year_dates = [('2020-12-31', '2020-12-31'), ('2021-12-31', '2021-12-31'), ('2022-12-31', '2022-12-31'),('2023-12-31', '2023-12-31'),('2024-12-31','2024-12-31'), ('2025-12-31','2025-12-31') ]
df['new_year'] = df['date'].apply(
    lambda x: 1 if any(pd.to_datetime(start) == x for start, end in new_year_dates) else 0)
#type region :
regions = {
    'Tunis': 'Hors côtier',
    'Sousse': 'Côtière',
    'Monastir': 'Côtière',
    'Sfax': 'Hors côtier',
    'Kairouan': 'Intérieure',
    'Gafsa': 'Intérieure',
    'Mahdia': 'Côtière',
    'Nabeul': 'Côtière',
    'Tozeur': 'Intérieure',
    'Jendouba': 'Intérieure',
    'Bizerte': 'Côtière',
    'Kasserine': 'Intérieure',
    'Tataouine': 'Intérieure',
    'Ariana':'Hors côtier',
    'Ben Arous':'Hors côtier',
    'Manouba':'Hors côtier',
    'Zaghouan':'Hors côtier',
    'Kébili':'Intérieure',
    'Gabès':'Intérieure',
    'Medenine':'Intérieure',
    'Sidi Bouzid':'Intérieure',
    'Beja':'Intérieure',
    'Kef':'Intérieure',
    'Siliana':'Intérieure'

}

df['Region'] = df['city'].apply(lambda x: regions.get(x, 'Unknown'))
# Définition des mois qui représentent la haute et basse saison touristique
high_season_months = [5,6,7,8]
medium_season_months = [4,9,10,11]
low_season_months = [1,2,3,12]

# Ajout d'une nouvelle colonne qui définit la saison touristique en fonction du mois
def get_season(month):
    if month in high_season_months:
        return 'Haute '
    elif month in medium_season_months:
        return 'Moyenne'
    else:
        return 'Basse '
df['saison_toristique'] = df['mois'].apply(lambda x: get_season(x))




In [ ]:
# Sauvegarder le DataFrame en fichier CSV
df.to_csv('features.csv', index=False)

# 3️⃣**📊 Regroupement des données par jour et calcul des indices météorologiques 🌦️**

Dans cette étape, nous avons regroupé les données de notre dataset pour obtenir des informations agrégées sur une base quotidienne. Nous avons aussi créé plusieurs indices météorologiques importants, permettant d'analyser des phénomènes extrêmes comme les vagues de chaleur, les vagues de froid, les tempêtes, les sécheresses, et les pluies intenses.

In [ ]:
'''Regrouper les données par jour'''
import pandas as pd

df1['date'] = pd.to_datetime(df1['date'])

# Regroupement par jour
df_journaliere = df1.groupby('date').agg({
    'temp': 'mean',  # Moyenne de la température
    'rhum': 'mean',  # Moyenne de l'humidité
    'wspd': 'mean',  # Moyenne de la vitesse du vent
    'prcp': 'sum',  # Somme des précipitations
    'dwpt': 'mean'  # Point de rosée moyen
}).reset_index()

# Renommage des colonnes pour éviter toute confusion
df_journaliere.rename(columns={
    'temp': 'temp_jour',
    'rhum': 'humidity_jour',
    'wspd': 'wind_speed_jour',
    'prcp': 'precip_jour',
    'dwpt': 'point de rosée/jour'
}, inplace=True)

print(df_journaliere.head())



        date  temp_jour  humidity_jour  wind_speed_jour  precip_jour  \
0 2020-02-24  13.800746      52.716418         7.581343          0.0   
1 2020-02-25  14.955556      49.996357         9.367395          0.0   
2 2020-02-26  15.680470      51.254973        11.401630          0.0   
3 2020-02-27  14.884991      55.578662        11.785172          0.0   
4 2020-02-28  15.732051      51.912088        12.672711          0.0   

   point de rosée/jour  
0             3.512687  
1             3.074499  
2             4.369982  
3             5.290416  
4             4.512821  


In [ ]:
print(df_journaliere.shape)

(1826, 9)


**Indice de Vague de Chaleur (Heat Wave Index)** : Cet indice détecte les vagues de chaleur en vérifiant si la température quotidienne dépasse 35°C pendant 3 jours consécutifs. L'indice prend la valeur 1 si la condition est remplie, sinon 0.

**Indice de Vague de Froid (Cold Wave Index)** : Cet indice identifie les vagues de froid en vérifiant si la température quotidienne reste inférieure à 9°C pendant 3 jours consécutifs.

**Indice de Pluie Intense (Heavy Rain Index)** : Cet indice signale une pluie intense lorsque les précipitations quotidiennes dépassent 50 mm.

**Indice de Tempête (Storm Index)** : Cet indice identifie les tempêtes en vérifiant si la vitesse du vent dépasse 60 km/h.

**Indice de Sécheresse (Drought Index)** : Cet indice détecte les périodes de sécheresse en vérifiant si les précipitations cumulées sur les 20 derniers jours sont inférieures à 10 mm.

In [ ]:
'''indice de Vague de Chaleur (Heat Wave Index)'''
df_journaliere['Indice de Vague de Chaleur'] = ((df_journaliere['temp_jour'] > 35).rolling(window=3).sum() >= 3).astype(int)
'''Indice de Vague de Froid (Cold Wave Index)'''
df_journaliere['Indice de Vague de Froid '] = ((df_journaliere['temp_jour'] < 9).rolling(window=3).sum() >= 3).astype(int)
'''Indice de Pluie Intense (Heavy Rain Index)'''
df_journaliere['Indice de Pluie Intense'] = (df_journaliere['precip_jour'] > 50).astype(int)
'''Indice de Tempête (Storm Index)'''
df_journaliere['Indice de Tempête'] = (df_journaliere['wind_speed_jour'] > 60).astype(int)
'''Indice de Sécheresse (Drought Index)'''
df_journaliere['Indice de sécheresse'] = (df_journaliere['precip_jour'].rolling(window=20).sum() < 10).astype(int)

In [ ]:
# Remplacer les valeurs NaN par 0
df_journaliere.fillna(0, inplace=True)

In [ ]:
pip install pythermalcomfort #Cette bibliothèque permet de calculer des indices de confort thermique, ce qui peut être utile pour analyser des conditions environnementales liées à la température, l'humidité, la vitesse du vent, etc.


  Obtaining dependency information for pythermalcomfort from https://files.pythonhosted.org/packages/36/7b/7c9f7be8248d048cef535acea7ab71fa9d23e8b7a2dba1ffa9b66e5315d9/pythermalcomfort-3.0.0-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/162.8 kB ? eta -:--:--
   -- ------------------------------------- 10.2/162.8 kB ? eta -:--:--
   ------- ------------------------------- 30.7/162.8 kB 435.7 kB/s eta 0:00:01
   ------------------- ------------------- 81.9/162.8 kB 573.4 kB/s eta 0:00:01
   ---------------------------- --------- 122.9/162.8 kB 654.9 kB/s eta 0:00:01
   ----------------------------------- -- 153.6/162.8 kB 654.6 kB/s eta 0:00:01
   ----------------------------------- -- 153.6/162.8 kB 654.6 kB/s eta 0:00:01
   ----------------------------------- -- 153.6/162.8 kB 654.6 kB/s eta 0:00:01
   ----------------------------------- -- 153.6/162.8 kB 654.6 kB/s eta 0:00:01
   -


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: C:\Users\salma souissi\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [ ]:
'''Indice climatique thermique universel (ICTU) '''
from pythermalcomfort.models import utci

# Calcul de l'ICTU simplifié pour chaque ligne
df_journaliere['Indice climatique thermique universel (ICTU)'] = df_journaliere.apply(
    lambda row: utci(tdb=row['temp_jour'], rh=row['humidity_jour'], v=row['wind_speed_jour'], tr=row['temp_jour']),
    axis=1
)

print(df_journaliere)

           date  temp_jour  humidity_jour  wind_speed_jour  precip_jour  \
0    2020-02-24  13.800746      52.716418         7.581343          0.0   
1    2020-02-25  14.955556      49.996357         9.367395          0.0   
2    2020-02-26  15.680470      51.254973        11.401630          0.0   
3    2020-02-27  14.884991      55.578662        11.785172          0.0   
4    2020-02-28  15.732051      51.912088        12.672711          0.0   
...         ...        ...            ...              ...          ...   
1821 2025-02-18  13.087326      75.267361         9.447743          0.0   
1822 2025-02-19  13.060243      78.616319         9.771007          0.0   
1823 2025-02-20  13.786806      77.914931        12.419792          1.4   
1824 2025-02-21  13.944097      78.774306        11.956597         13.3   
1825 2025-02-22  14.126620      71.662037        11.531713          2.8   

      point de rosée/jour  Indice de Chaleur  refroidissement éolien  \
0                3.512687  

1. **Heat Index (Indice de Chaleur)** : L'indice de chaleur est calculé en fonction de la température et de l'humidité relative. Si la température est inférieure à 27°C ou si l'humidité relative est inférieure à 40%, il retourne simplement la température. Sinon, il utilise une formule plus complexe qui tient compte de ces deux paramètres pour donner une estimation de la sensation thermique.

2. **Wind Chill (Refroidissement Éolien)** : Le refroidissement éolien est utilisé pour décrire la sensation thermique ressentie en raison du vent. Si la température est supérieure à 10°C ou si la vitesse du vent est inférieure à 4,8 km/h, la température réelle est utilisée. Sinon, une formule prenant en compte la température et la vitesse du vent calcule un refroidissement supplémentaire dû au vent.

3. **Humidex (Indice d’Inconfort)** : L'humidex est un indice utilisé pour quantifier l'inconfort ressenti à cause de la chaleur et de l'humidité. Il utilise la température et le point de rosée pour calculer une valeur représentant le confort thermique.

In [ ]:
import pandas as pd
import numpy as np

# Fonctions de calcul des indices de confort thermique
'''Heat Index (Indice de Chaleur)'''

def calculate_heat_index(temp, rhum):
    if temp < 27 or rhum < 40:
        return temp
    else:
        return (
            -8.78469475556 + 1.61139411 * temp + 2.33854883889 * rhum
            - 0.14611605 * temp * rhum - 0.012308094 * (temp ** 2)
            - 0.0164248277778 * (rhum ** 2) + 0.002211732 * (temp ** 2) * rhum
            + 0.00072546 * temp * (rhum ** 2) - 0.000003582 * (temp ** 2) * (rhum ** 2)
        )
'''refroidissement éolien'''
def calculate_wind_chill(temp, wspd):
    if temp > 10 or wspd < 4.8:
        return temp
    else:
        return 13.12 + 0.6215 * temp - 11.37 * (wspd ** 0.16) + 0.3965 * temp * (wspd ** 0.16)
'''Indice d’inconfort'''
def calculate_humidex(temp, dwpt):
    if temp < 20 or dwpt < 10:
        return temp
    else:
        return temp + 0.5555 * (6.11 * np.exp(5417.7530 * (1/273.16 - 1/(dwpt + 273.15))) - 10)

# Ajout des colonnes des indices
df_journaliere['Indice de Chaleur'] = df_journaliere.apply(lambda row: calculate_heat_index(row['temp_jour'], row['humidity_jour']), axis=1)
df_journaliere['refroidissement éolien'] = df_journaliere.apply(lambda row: calculate_wind_chill(row['temp_jour'], row['wind_speed_jour']), axis=1)
df_journaliere['Indice d’inconfort'] = df_journaliere.apply(lambda row: calculate_humidex(row['temp_jour'], row['point de rosée/jour']), axis=1)

# Affichage du DataFrame mis à jour
print(df_journaliere)

           date  temp_jour  humidity_jour  wind_speed_jour  precip_jour  \
0    2020-02-24  13.800746      52.716418         7.581343          0.0   
1    2020-02-25  14.955556      49.996357         9.367395          0.0   
2    2020-02-26  15.680470      51.254973        11.401630          0.0   
3    2020-02-27  14.884991      55.578662        11.785172          0.0   
4    2020-02-28  15.732051      51.912088        12.672711          0.0   
...         ...        ...            ...              ...          ...   
1821 2025-02-18  13.087326      75.267361         9.447743          0.0   
1822 2025-02-19  13.060243      78.616319         9.771007          0.0   
1823 2025-02-20  13.786806      77.914931        12.419792          1.4   
1824 2025-02-21  13.944097      78.774306        11.956597         13.3   
1825 2025-02-22  14.126620      71.662037        11.531713          2.8   

      point de rosée/jour  Indice de Chaleur  refroidissement éolien  \
0                3.512687  

In [ ]:
# Remplacer les valeurs NaN par 0
df_journaliere.fillna(0, inplace=True)

In [ ]:
print(df_journaliere.shape)

(1826, 15)


In [ ]:
import pandas as pd
df = pd.read_csv("features.csv")

In [ ]:
# Fusionner les DataFrames  sur la colonne 'date'
df['date'] = pd.to_datetime(df['date'])

df = df.merge(df_journaliere, on='date', how='left')


print(df.head())

    city       date  hour  temperature  precipitation  wind_speed  humidity  \
0  Tunis 2020-02-24    18         16.3            0.0         7.6      39.0   
1  Tunis 2020-02-24    19         15.0            0.0        13.0      39.0   
2  Tunis 2020-02-24    20         15.0            0.0        11.2      33.0   
3  Tunis 2020-02-24    21         14.1            0.0         1.8      35.0   
4  Tunis 2020-02-24    22         13.0            0.0         9.4      44.0   

   température_moyenne_7j  mois      Jour  ...  point de rosée/jour  \
0               16.300000     2     Lundi  ...             3.512687   
1               15.650000     2     Mardi  ...             3.512687   
2               15.433333     2  Mercredi  ...             3.512687   
3               15.100000     2     Jeudi  ...             3.512687   
4               14.680000     2  Vendredi  ...             3.512687   

   Indice de Chaleur refroidissement éolien  Indice d’inconfort  \
0          13.800746           

In [ ]:

df.to_csv('features.csv', index=False)

In [ ]:
print(df.columns)

Index(['city', 'date', 'hour', 'temperature', 'precipitation', 'wind_speed',
       'humidity', 'température_moyenne_7j', 'mois', 'Jour', 'Weekend',
       'jour_ferie', 'saison', 'vacances_scolaires', 'ramadan', 'new_year',
       'Region', 'saison_toristique', 'periode_journee', 'ventes_glaces',
       'ventes_boissons', 'temp_jour', 'humidity_jour', 'wind_speed_jour',
       'precip_jour', 'point de rosée/jour', 'Indice de Chaleur',
       'refroidissement éolien', 'Indice d’inconfort',
       'Indice de Vague de Chaleur', 'Indice de Vague de Froid ',
       'Indice de Pluie Intense', 'Indice de Tempête', 'Indice de sécheresse',
       'Indice climatique thermique universel (ICTU)'],
      dtype='object')


# 4️⃣ **Ajouter des variables de décalage temporel (ou lag features)**

In [ ]:
'''Ajout de variables de décalage temporel (Lag Features)'''
# Nombre de jours de décalage ( 1, 3, 7 jours)
lags = [1, 3, 7]

for lag in lags:
    df[f'temp_jour_lag{lag}'] = df['temp_jour'].shift(lag)
    df[f'humidity_jour_lag{lag}'] = df['humidity_jour'].shift(lag)
    df[f'wind_speed_jour_lag{lag}'] = df['wind_speed_jour'].shift(lag)
    df[f'precip_jour_lag{lag}'] = df['precip_jour'].shift(lag)

print(df.head())



    city        date  hour  temperature  precipitation  wind_speed  humidity  \
0  Tunis  2020-02-24    18         16.3            0.0         7.6      39.0   
1  Tunis  2020-02-24    19         15.0            0.0        13.0      39.0   
2  Tunis  2020-02-24    20         15.0            0.0        11.2      33.0   
3  Tunis  2020-02-24    21         14.1            0.0         1.8      35.0   
4  Tunis  2020-02-24    22         13.0            0.0         9.4      44.0   

   mois      Jour  Weekend  ...  wind_speed_jour_lag1 temp_jour_lag3  \
0     2     Lundi        0  ...                   NaN            NaN   
1     2     Mardi        0  ...              7.581343            NaN   
2     2  Mercredi        0  ...              7.581343            NaN   
3     2     Jeudi        0  ...              7.581343      13.800746   
4     2  Vendredi        0  ...              7.581343      13.800746   

   humidity_jour_lag3  wind_speed_jour_lag3  temp_jour_lag7  \
0                 NaN  

In [ ]:
# Remplacer les valeurs NaN par 0
df.fillna(0, inplace=True)

In [ ]:
'''Supprimer les heures en dehors des horaires d’ouverture'''
df = df[df['hour'].between(7, 23) | (df['hour'] == 0)] #garde les heures entre 7h et 23h ,00h


In [ ]:
print(df.shape)

(782170, 44)


In [ ]:
print(df.columns)

Index(['city', 'date', 'hour', 'temperature', 'precipitation', 'wind_speed',
       'humidity', 'mois', 'Jour', 'Weekend', 'jour_ferie', 'saison',
       'vacances_scolaires', 'ramadan', 'new_year', 'Region',
       'saison_toristique', 'periode_journee', 'temp_jour', 'humidity_jour',
       'wind_speed_jour', 'precip_jour', 'point de rosée/jour',
       'Indice de Chaleur', 'refroidissement éolien', 'Indice d’inconfort',
       'Indice de Vague de Chaleur', 'Indice de Vague de Froid ',
       'Indice de Pluie Intense', 'Indice de Tempête', 'Indice de sécheresse',
       'Indice climatique thermique universel (ICTU)', 'temp_jour_lag1',
       'humidity_jour_lag1', 'wind_speed_jour_lag1', 'temp_jour_lag3',
       'humidity_jour_lag3', 'wind_speed_jour_lag3', 'temp_jour_lag7',
       'humidity_jour_lag7', 'wind_speed_jour_lag7', 'precip_jour_lag1',
       'precip_jour_lag3', 'precip_jour_lag7'],
      dtype='object')


In [ ]:
# reorganisation des colonnes
ordered_columns = [
    # 1. Informations de base
    'city', 'Region', 'date', 'hour',

    # 2. Données météorologiques horaires
    'temperature', 'precipitation', 'wind_speed', 'humidity',

    # 3. Données météorologiques journalières
    'temp_jour', 'humidity_jour', 'wind_speed_jour', 'precip_jour', 'point de rosée/jour',
    # 4. Variables de décalage temporel (lags)
    'temp_jour_lag1', 'humidity_jour_lag1', 'wind_speed_jour_lag1','precip_jour_lag1' ,
    'temp_jour_lag3', 'humidity_jour_lag3', 'wind_speed_jour_lag3', 'precip_jour_lag3',
    'temp_jour_lag7', 'humidity_jour_lag7', 'wind_speed_jour_lag7','precip_jour_lag7',
    # 5. Données contextuelles
    'mois', 'Jour', 'Weekend', 'jour_ferie', 'saison', 'vacances_scolaires',
    'ramadan', 'new_year', 'saison_toristique', 'periode_journee',

    # 6. Indices météorologiques
    'Indice de Chaleur', 'refroidissement éolien', 'Indice d’inconfort', 'Indice de Vague de Chaleur',
    'Indice de Vague de Froid ', 'Indice de Pluie Intense', 'Indice de Tempête', 'Indice de sécheresse',
    'Indice climatique thermique universel (ICTU)',

]

# Réorganiser le dataframe
df = df[ordered_columns]


In [ ]:

df.to_csv('features.csv', index=False)

In [ ]:
''' convertir toutes les colonnes de type float en int'''
df = df.round().astype({col: 'int' for col in df.select_dtypes('float').columns})


In [ ]:

df.to_csv('features.csv', index=False)

In [ ]:
import pandas as pd
df=pd.read_csv('features.csv')

# 5️⃣ **Catégorisation des indices de confort thermique**

In [ ]:
import re


df['Indice climatique thermique universel (ICTU)'] = df['Indice climatique thermique universel (ICTU)'].apply(
    lambda x: re.search(r"stress_category='([^']*)'", str(x)).group(1) if x is not None and "stress_category=" in str(x) else None
)


In [ ]:
print(df['Indice climatique thermique universel (ICTU)'].unique())

['slight cold stress' 'moderate cold stress' 'unknown'
 'strong cold stress' 'no thermal stress' 'moderate heat stress'
 'strong heat stress']


In [ ]:
import pandas as pd
df=pd.read_csv('features.csv')
# Convertir les colonnes de string en float ou int
df['Indice de Chaleur'] = pd.to_numeric(df['Indice de Chaleur'], errors='coerce')
df['refroidissement éolien'] = pd.to_numeric(df['refroidissement éolien'], errors='coerce')
df['Indice d’inconfort'] = pd.to_numeric(df['Indice d’inconfort'], errors='coerce')
def categoriser_indice_chaleur(valeur):
    if valeur <= 10:
        return "Frais"
    elif valeur <= 15:
        return "Confortable frais"
    elif valeur <= 20:
        return "Confortable"
    elif valeur <= 25:
        return "Légèrement chaud"
    elif valeur <= 29:
        return "Chaud"
    elif valeur <= 34:
        return "Très chaud"
    else:
        return "Extrêmement chaud"

df['Indice de Chaleur'] = df['Indice de Chaleur'].apply(categoriser_indice_chaleur)
def categoriser_refroidissement_eolien(valeur):
    if valeur <= 8:
        return "Froid"
    elif valeur <= 13:
        return "Frais"
    elif valeur <= 18:
        return "Modéré"
    elif valeur <= 23:
        return "Confortable"
    elif valeur <= 28:
        return "Agréablement chaud"
    elif valeur <= 33:
        return "Chaud"
    else:
        return "Très chaud"

df['refroidissement éolien'] = df['refroidissement éolien'].apply(categoriser_refroidissement_eolien)
def categoriser_indice_inconfort(valeur):
    if valeur <= 12:
        return "Frais à froid"
    elif valeur <= 18:
        return "Confortable frais"
    elif valeur <= 23:
        return "Confortable optimal"
    elif valeur <= 27:
        return "Légèrement inconfortable"
    elif valeur <= 31:
        return "Inconfortable"
    elif valeur <= 36:
        return "Très inconfortable"
    else:
        return "Extrêmement inconfortable"

df['Indice d’inconfort'] = df['Indice d’inconfort'].apply(categoriser_indice_inconfort)

In [ ]:
print(df['Indice de Chaleur'].unique())
print(df['refroidissement éolien'].unique())
print(df['Indice d’inconfort'].unique())

['Confortable frais' 'Confortable' 'Frais' 'Légèrement chaud' 'Chaud'
 'Très chaud' 'Extrêmement chaud']
['Modéré' 'Frais' 'Confortable' 'Agréablement chaud' 'Chaud' 'Froid'
 'Très chaud']
['Confortable frais' 'Frais à froid' 'Confortable optimal'
 'Légèrement inconfortable' 'Inconfortable' 'Très inconfortable'
 'Extrêmement inconfortable']


In [ ]:
df=df.to_csv('features.csv')

In [ ]:
df=pd.read_csv('features.csv')

In [ ]:
import pandas as pd
print(df.shape)

(782170, 45)


#  **Conclusion finale**
---
***Ce notebook a permis de transformer un simple dataset météorologique en une base de données riche et exploitable grâce à des features avancées. Ces nouvelles informations ouvrent la porte à des analyses plus précises et pertinentes, aide ainsi les commerçants à optimiser leur gestion des stocks et leurs stratégies marketing en
fonction de la météo.
.***